# Indexing blog posts for a blog roll

This notebooks aggregates the `deathbeds` modules into a tidy `pandas.DataFrame`.  That object can be used to created a structured view of the posts.

In [1]:
    from pathlib import Path
    import deathbeds
    from pandas import DataFrame, Series, Index, to_datetime
    from IPython.display import Markdown, display
    loader = __import__('importnb').Execute(stdout=True, stderr=True, display=True);

* glob the modules in the `deathbeds` package.

In [2]:
    modules = Series(index=Index(Path(deathbeds.__file__).parent.glob('*.ipynb')));

Convert a file path to and imported module

In [ ]:
    def file_to_module(path):
        with loader:
            name = path.relative_to(Path(deathbeds.__file__).parent).stem.split('.', 1)[0]
            return getattr(__import__('.'.join(('deathbeds', name))), name)

* Import all of the modules.  As the modules are imported we are testing their efficacy when `__name__ != '__main__`

In [ ]:
    if __name__ == '__main__':
        for path in modules.index: 
            try: modules.loc[path] = file_to_module(path)
            except BaseException as e: print(path, e)

* Tidy the module information to create a blog roll

In [ ]:
    if __name__ == '__main__':
        df = modules.to_frame('module')
        df['docstring'] = df['module'].apply(lambda x: x.__doc__)
        df['name'] = df['module'].apply(lambda x: x.__name__)
        df['title'] = df.docstring.str.lstrip('#').str.lstrip().apply(lambda x: x.splitlines()[0])
        df['relative'] = df.index.map(lambda x: f'{x.relative_to(Path(deathbeds.__file__).parent)}')

        df['date'] = df['name'].str.split('.', 1).apply(lambda _: '-'.join(_[1].split('-', 3)[:3])).pipe(
            lambda s: s[s.apply(lambda x: x[0]).str.isnumeric()]
        ).pipe(to_datetime)


        blog_roll = Markdown(df.dropna(subset=['date']).sort_values('date', ascending=False).apply(
            lambda _: f"""* ## [{_.loc['title']}]({_.loc['relative']})

    > {_.loc['date'].strftime("%A  %B %d, %Y")}

    """, axis=1
        ).pipe("".join));

In [ ]:
    if __name__ == '__main__':
        display(df.sample(2).reset_index(drop=True))

In [ ]:
    if __name__ == '__main__': 
        display(blog_roll)

## Infer the `deathbeds` requirements.

All of the modules have been imported into the current context.  That means with a few logic circuits we could recreate the requirements for all of the notebooks in `deathbeds`.

* `mappings` renames import names to their package name.

In [ ]:
import sys

In [ ]:
    mappings = dict(
        PIL='pillow',
        skimage='scikit-image', 
        yaml= 'pyyaml',
        depfinder="git+https://github.com/deathbeds/depfinder@ipython-code-transform",
        **{
            'cv2.cv2': 'opencv-python', 
        }
    );

* there are some [Jupyter-console incompatible with prompt-toolkit 2.0.2 #158](https://github.com/jupyter/jupyter_console/issues/158)

In [ ]:
    mappings.update(prompt_toolkit='prompt-toolkit==1.0.15')

* some of the imports we flat out don't want.

In [ ]:
    nah = 'sphinxcontrib', 'pkg_resources', 'dateutil', 'storemagic', 'OpenSSL', 'socks', 'gmpy2', 'blib2to3', 'deathbeds';

* special case

In [ ]:
    special = "cv2.cv2",;

### Logic

* The module has a `__file__` attribute; builtins don't
* The module is top level; it doesn't have a `.`
* The module name doesn't start with an `_`
* The module is install `install` or `develop`; in `site-packages` or the working directories.

In [ ]:
    requirements = ""

    for key, module in __import__('sys').modules.items():
        file = getattr(module, '__file__', "")
        if module.__name__ in special or (
            '.' not in key 
            and '.' not in module.__name__
            and not key.startswith('_')
            and (
                all(map(file.__contains__, ('python', 'site-packages')))
                or 'python' not in getattr(module, '__file__', "python")
            ) and module.__name__ not in nah
        ):
            requirements += f"""{mappings.get(module.__name__, module.__name__)}\n"""

    requirements = '\n'.join(sorted(requirements.splitlines(), key=str.lower));
    Path('../requirements.txt').write_text(requirements)
    print(requirements)